In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
from langchain_core.documents import Document
from langchain.tools import tool
from langchain_openai import ChatOpenAI

In [23]:
animal_facts = {
    "강아지": "강아지는 인간과 가장 친한 반려동물 중 하나입니다. 충성심이 강하고 활발합니다.",
    "고양이": "고양이는 독립적인 성격을 가진 반려동물로, 깨끗하고 조용한 성격을 가졌습니다.",
    "코끼리": "코끼리는 육상 동물 중 가장 큰 동물로, 매우 높은 지능과 사회성을 지닙니다.",
    "기린": "기린은 목이 매우 길며, 주로 아프리카 사바나에서 서식합니다."
}

In [24]:
@tool
def search_animal(query: str) ->list[Document]:
    """
    이름에 해당하는 동물 정보를 간단히 반환합니다.
    """
    if query in animal_facts:
        return [Document(page_content=animal_facts[query])]
    return [Document(page_content="해동 동물 정보를 찾을 수 없습니다.")]

In [25]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain, RunnableConfig

today = datetime.today().strftime('%Y-%m-%d')

prompt = ChatPromptTemplate([
    ('system', f"You are helpful AI assistant. Today's date is {today}."),
    ('human', '{user_input}'),
    ('placeholder', '{message}')
])

llm = ChatOpenAI(model='gpt-4o-mini')
llm_with_tools = llm.bind_tools(tools=[search_animal])
llm_chain = prompt | llm_with_tools

@chain
def web_search_chain(user_input: str):
    input_ = {'user_input': user_input}
    ai_msg = llm_chain.invoke(input_)

    tool_msg = search_animal.batch(ai_msg.tool_calls)

    return llm_chain.invoke({**input_, "message": [ai_msg, *tool_msg]})

In [26]:
question = '고양이에 대해 알려줘'
response = web_search_chain.invoke(question)
print('\n 최종 응답:\n', response.content)


 최종 응답:
 고양이는 독립적인 성격을 가진 반려동물로, 깨끗하고 조용한 성격을 가지고 있습니다.
